In [2]:
%load_ext autoreload
%autoreload
from feedgen.feed import FeedGenerator
import boto3
import gzip
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-1')
table = dynamodb.Table('RedditEntry')


In [35]:
# doneが1でないものを取得
res = table.scan(
    ProjectionExpression="entry_url, tag, title, published, author, rss_write_time",
    FilterExpression=boto3.dynamodb.conditions.Attr('done').ne(1),
    ReturnConsumedCapacity='TOTAL',
)


1707482513

In [62]:
current_from_epoch = int(datetime.now().strftime('%s'))

# rss_write_timeがない、または2日以内のものを取得する。
limit_from_epoch = current_from_epoch - 60 * 60 * 24 * 2
res = table.scan(
    ProjectionExpression="entry_url, tag, title, published, author, rss_write_time",
    FilterExpression="attribute_not_exists(rss_write_time) OR rss_write_time > :limit_from_epoch",
    ExpressionAttributeValues={":limit_from_epoch": limit_from_epoch},
    ReturnConsumedCapacity='TOTAL',
)


In [63]:
res

{'Items': [{'entry_url': 'https://www.reddit.com/r/playmygame/comments/1amdtrg/public_beta_for_kronia_is_now_available/',
   'published': '2024-02-09T02:51:17+00:00',
   'author': '/u/REALREALBlockManBlue',
   'rss_write_time': Decimal('1707482868'),
   'title': 'Public Beta for Kronia is now available!'},
  {'entry_url': 'https://www.reddit.com/r/unity/comments/1amjj2w/cannot_uncheck_script/',
   'published': '2024-02-09T08:27:10+00:00',
   'author': '/u/DylanDahmer',
   'rss_write_time': Decimal('1707482868'),
   'title': 'Cannot uncheck script'},
  {'entry_url': 'https://www.reddit.com/r/unity/comments/1aml8om/buying_an_asset/',
   'published': '2024-02-09T10:32:57+00:00',
   'author': '/u/WeekIcy7669',
   'rss_write_time': Decimal('1707482868'),
   'title': 'Buying an asset'},
  {'entry_url': 'https://www.reddit.com/r/playmygame/comments/1also1n/i_made_a_fantasy_rpg_demo_in_rpg_maker_mv/',
   'published': '2024-02-08T10:24:34+00:00',
   'author': '/u/Hopeful_Candle_4479',
   'rss_w

In [32]:
fg = FeedGenerator()
fg.title("Rexxit Feed")
fg.link(href="https://www.reddit.com")
fg.description(" ")

for entry in res['Items']:
    # print("title", entry['title'])
    # print(entry['entry_url'])
    # print("published", entry['published'])
    # print("author", entry['author'])
    # print('---')
    fe = fg.add_entry()
    # fe.id(entry['entry_url'])
    fe.title(f"[{entry.get('tag', 'none')}] {entry['title']}")
    # pubdate = datetime.strptime(audio.name.split("__")[0], "%Y%m%d_%H%M")
    # pubdate = pubdate.astimezone(datetime.now().astimezone().tzinfo)
    fe.published(entry['published'])
    fe.link(href=entry['entry_url'])
    fe.description(" ")


In [33]:
filecontent = fg.rss_str()
filecontent

b'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<rss xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" version="2.0"><channel><title>Rexxit Feed</title><link>https://www.reddit.com</link><description> </description><docs>http://www.rssboard.org/rss-specification</docs><generator>python-feedgen</generator><lastBuildDate>Fri, 09 Feb 2024 12:34:02 +0000</lastBuildDate><item><title>[none] Seeking Beta playtesters for a new shooter game</title><link>https://www.reddit.com/r/playmygame/comments/1aluvyi/seeking_beta_playtesters_for_a_new_shooter_game/</link><description> </description><pubDate>Thu, 08 Feb 2024 12:45:02 +0000</pubDate></item><item><title>[none] Before &amp; After post! Project Mix\'s visuals in early 2022 vs end of 2023. We\'re constantly refining the artstyle of the game! What do you think?</title><link>https://www.reddit.com/r/Unity3D/comments/1amffk8/before_after_post_project_mixs_visuals_in_early/</link><description> </descript

In [34]:
# s3へ保存する。
s3 = boto3.resource('s3')
bucket = s3.Bucket('myfeeds-jh2694gn')
bucket.put_object(Key='public/rexxit.rss', Body=filecontent, ContentType='application/rss+xml')

s3.Object(bucket_name='myfeeds-jh2694gn', key='public/rexxit.rss')

In [57]:
"aa" in res['Items'][0]

False

In [59]:
from datetime import datetime
current_from_epoch = int(datetime.now().strftime('%s'))
for entry in res['Items']:
    if "rss_write_time" in entry:
        continue
    table.update_item(
        Key={'entry_url': entry["entry_url"]},
        UpdateExpression='SET rss_write_time = :rss_write_time',
        ConditionExpression='attribute_exists(entry_url) AND attribute_not_exists(rss_write_time)',
        ExpressionAttributeValues={
            ':rss_write_time': current_from_epoch,
        },
    )

In [20]:
for entry in res['Items']:
    print("title", entry['title'])
    print(entry['entry_url'])
    print("published", entry['published'])
    print("author", entry['author'])
    print('---')

title Public Beta for Kronia is now available!
https://www.reddit.com/r/playmygame/comments/1amdtrg/public_beta_for_kronia_is_now_available/
published 2024-02-09T02:51:17+00:00
author /u/REALREALBlockManBlue
---
title Cannot uncheck script
https://www.reddit.com/r/unity/comments/1amjj2w/cannot_uncheck_script/
published 2024-02-09T08:27:10+00:00
author /u/DylanDahmer
---
title Buying an asset
https://www.reddit.com/r/unity/comments/1aml8om/buying_an_asset/
published 2024-02-09T10:32:57+00:00
author /u/WeekIcy7669
---
title I made a Fantasy RPG Demo in RPG Maker MV
https://www.reddit.com/r/playmygame/comments/1also1n/i_made_a_fantasy_rpg_demo_in_rpg_maker_mv/
published 2024-02-08T10:24:34+00:00
author /u/Hopeful_Candle_4479
---
title As an indie game developer, I consistently experience a mix of love and frustration towards my work. I would like to share this project that I have been diligently working on for the past four months.
https://www.reddit.com/r/gamedev/comments/1amjx4v/as_an_i